# 🔥 **ROSHAN Interactive Usage Notebook**
### *A Simple Guide to Configuring and Running ROSHAN*

**ROSHAN** is a simulation and reinforcement-learning framework for autonomous wildfire suppression using UAV agents.  
It combines a high-fidelity C++ wildfire simulator with a Python-based hierarchical learning architecture. Although the internal system is complex, using ROSHAN is straightforward: nearly all behavior is controlled through a single YAML configuration file that specifies what to train, how to train it, and under which conditions simulations are executed.

This notebook provides a lightweight interface for working with ROSHAN without needing to modify any internal code. Its purpose is to help users understand the configuration file, adjust selected parameters, and execute training or evaluation runs from a unified environment. It offers a practical, accessible entry point into ROSHAN without requiring any knowledge of its underlying implementation details.

The workflow in this notebook is simple:

1. **Load the main ROSHAN configuration file.**  
   This file governs almost every aspect of the system.

2. **Explain the core components of the configuration.**  
   These include:
   - **Hierarchy level** to train or evaluate (FlyAgent, ExploreAgent, PlannerAgent)  
   - **Algorithmic hyperparameters** (learning rate, discount factors, PPO settings)  
   - **Agent hyperparameters** (movement limits, observation configuration, rewards, etc.)  
   - **General settings** (experiment seeds, environment options, logging paths, etc.)

3. **Allow the user to modify selected configuration values.**  
   Only parameters that are meaningful and safe to change from the notebook are exposed.  
   More specialized aspects—like internal network architectures—must be changed in the source code, and ROSHAN automatically records architecture changes alongside the model weights.

4. **Generate an execution-ready config file** and **run the simulation or training** directly from the notebook.  
   The C++ simulation is invoked from Python, fully abstracted away, ensuring the system behaves consistently with the provided configuration.


### 📁 Load the Base ROSHAN Configuration

Before we modify or execute anything, we first load the main `notebook_config.yaml` file.  
This file contains all parameters used by ROSHAN during training or evaluation.

In [1]:
import yaml, os
from pprint import pprint
from src.pysim.utils import get_project_paths

root_path = get_project_paths("root_path")
config_path = os.path.join(root_path, 'notebook_config.yaml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
pprint(config)

{'algorithm': {'IQL': {'batch_size': 250,
                       'beta1': 0.9,
                       'beta2': 0.999,
                       'buffer_path': 'models/PPOCollisionBaseline_OptimizedReward_Top10Mean_Tanh/training_8/memory.pkl',
                       'discount': 0.99,
                       'expectile': 0.7,
                       'k_epochs': 1,
                       'lr': 0.0001575,
                       'offline_updates': 10,
                       'online_updates': 0,
                       'policy_freq': 50,
                       'tau': 0.005,
                       'temperature': 3.0},
               'PPO': {'_lambda': 0.98,
                       'batch_size': 4000,
                       'beta1': 0.9,
                       'beta2': 0.999,
                       'decay_rate': 0.9999,
                       'entropy_coeff': 0.0005,
                       'eps_clip': 0.2,
                       'gamma': 0.9,
                       'horizon': 120000,
                

### 📘 Understanding the ROSHAN Configuration Dictionary

Now that you have inspected the full configuration dictionary, this section provides a structured explanation of its core components.  
The goal is not to describe every value, but to give a clear understanding of how ROSHAN interprets the configuration and which parts matter for training or evaluation—especially when using **PPO** and working with the **FlyAgent** or **PlannerAgent** hierarchies.

The configuration is divided into several top-level sections:

---

## ⚙️ **1. `settings` — Global Behavior of the Framework**
This section defines the overall execution mode of ROSHAN.  
Important parameters include:

- **`mode`**: GUI/no-GUI mode. For training, this is normally `NoGUI_RL` since it is much faster.  
- **`seed`**: Controls reproducibility.  
- **`rl_mode`**: Whether the system runs in *training* or *evaluation* mode.  
- **`hierarchy_type`**: Selects which agent hierarchy to train or evaluate (`fly_agent`, `planner_agent`).  
- **`resume`**: Whether to continue from an existing checkpoint.  
- **`auto_train`**: Usually **disabled** in this notebook, but some of its parameters  
  - **`max_train`** and **`max_eval`** can still be used when running in NoGUI mode to
  determine how long training or evaluation should run before stopping automatically.
- **`optuna`**: Hyperparameter search; not relevant for manual training here.

These settings determine the *high-level control flow* of ROSHAN.

---

## 📁 **2. `paths` — Where Models are Stored and which Map is loaded**
This section defines how ROSHAN handles filesystem paths:

- **`model_directory`**: Where trained models will be saved or loaded from.  
- **`model_name`**: Which checkpoint to load (e.g., `"latest"`, `"best_reward"`).  
- **`init_map`**: Which terrain map is used for simulations in no-GUI mode.  

These values allow ROSHAN to store and retrieve models consistently.

---

## 🧠 **3. `algorithm` — Reinforcement Learning Settings**
ROSHAN supports several RL algorithms, but in this notebook we focus on **PPO**.  
Important PPO-related parameters include:

- **`share_encoder`**: Whether policy and value share a feature extractor.  
- **`use_tanh_dist`**: Whether actions use a TanhNormal distribution.  
- **`PPO` subsection**:
  - **`lr`**: Learning rate.  
  - **`batch_size`**, **`horizon`**: How much data is collected before updating.  
  - **`k_epochs`**: Number of update passes per batch.  
  - **`entropy_coeff`**: Controls exploration.  
  - **`value_loss_coef`**: Balances value vs. policy learning.  
  - **`gamma`**, **`_lambda`**: Discount and GAE parameters.  
  - **`eps_clip`**: PPO clipping range.  
  - Optional features: KL early stopping, manual log-std decay, separate optimizers.

These parameters define how PPO updates policies for both FlyAgent and PlannerAgent.

---

## 🔥 **4. `fire_model` — C++ Simulation Parameters**
This section configures the wildfire simulator.  
It includes cell properties, particle emission, wind conditions, and rendering.

For this notebook, **you normally do not modify these values**.  
They are used internally by the C++ environment and remain stable for training.

---

## 🌲 **5. `environment` — Fire Initialization and Agent Behavior**
This section defines how simulations start and how agents are placed:

- Initial fire distribution (`num_fire_clusters`, `fire_percentage`)  
- Agent behavior (`fire_goal_percentage`, starting positions)  
- Evaluation-only features (water limits, recharge time)

Again, these are rarely modified from the notebook.

---

## 🚁 **6. `environment.agent.fly_agent` — Low-Level Flight Controller**
This section configures the **FlyAgent**, which directly navigates and extinguishes fires.

Key parameters:

- **`collision`**: Whether the agent must avoid others.  
- **`use_simple_policy`**: Episode ends when reaching a goal or when fires are extinguished.  
- **`use_velocity_change` / `use_vel_bins`**: Defines the action representation.  
- **`frame_skips`**: How often actions are chosen.  
- **`algorithm`**: Set to `"PPO"` for learning.  
- **`num_agents`**: Number of parallel FlyAgents.  
- **`time_steps`**: Memory stack length for inputs.  
- **`max_speed`, `view_range`**: Movement and observation limits.  
- **Rewards**: Shaping terms like distance improvement, boundary penalty, extinguishing bonus, etc.

These parameters strongly influence FlyAgent learning behavior.

---

## 🧭 **7. `environment.agent.planner_agent` — High-Level Goal Allocation**
The PlannerAgent assigns extinguishing goals to low-level agents:

- **`hierarchy_timesteps`**: How many FlyAgent steps per planner step.  
- **`algorithm`**: Also uses `"PPO"`.  
- **`num_agents`**: Number of high-level planners.  
- **`view_range`, `max_speed`**: Passed to the internal low-level agents.  
- **`default_model_folder` & `default_model_name`**: Which FlyAgent model is used as the planner’s low-level controller.  
- **Rewards**: Focus on map-wide fire suppression, coordination penalties, and prioritization of meaningful goal selection.

The PlannerAgent is trained on a much higher abstraction level than the FlyAgent.

---

In this notebook, you will primarily modify:

- elements inside **`settings`**  
- important values in **`algorithm -> PPO`**  
- selected parameters inside **FlyAgent** or **PlannerAgent**

Everything else—especially simulation physics—is handled automatically by the system. 
## Now we take a closer look into the two hierarchy settings:

### 🚁 Configuring the `fly_agent`

The `fly_agent` represents ROSHAN’s **low-level controller**.  
It directly navigates the environment, avoids obstacles, and extinguishes fires.  
Below is a brief overview of the key parameters you may want to adjust.

---

#### 🛑 **`collision` — Collision-aware vs. collision-free training**
```python
config["environment"]["agent"]["fly_agent"]["collision"] = True
```
- When enabled, the FlyAgent must avoid other agents.
- This changes the network input space and terminal conditions.
#### 🎯 **`use_simple_policy` — Simple vs. complex episode structure**
```python
config["environment"]["agent"]["fly_agent"]["use_simple_policy"] = False
```
- **Simple policy**: Episode ends once a single agent reaches its goal.
- **Complex policy**: Episode ends when all fires are extinguished(best practice for collision tasks).
#### 🏃 **Action representation (`use_velocity_change`, `use_vel_bins`)**
```python
config["environment"]["agent"]["fly_agent"]["use_velocity_change"] = False
config["environment"]["agent"]["fly_agent"]["use_vel_bins"] = True
```
These options determine how actions are interpreted:
- use_velocity_change = True: Actions represent accelerations.
- use_velocity_change = False: Actions represent absolute velocities.
- use_vel_bins = True: Velocities are discretized (100 bins per dimension),making training more stable.

### 🧭 Configuring the `planner_agent`

The `planner_agent` represents ROSHAN’s **high-level decision maker**. Unlike the FlyAgent—which handles local movement and extinguishing—the Planner assigns **goals** to a group of low-level agents and operates on a slower timescale.

Below is a brief explanation of the key configuration parameters you may want to adjust when training or evaluating the Planner.

---

#### 🔄 **`hierarchy_timesteps` — How often the Planner acts**
```python
config["environment"]["agent"]["planner_agent"]["hierarchy_timesteps"] = 3
```
This parameter determines how many low-level FlyAgent steps occur for every one Planner step.

- A higher value means the Planner acts less frequently, giving FlyAgents more time to execute each assigned goal.
- A lower value means the Planner makes decisions more frequently, leading to finer but potentially noisier coordination.

This value strongly influences how “strategic” or “reactive” the Planner becomes.

#### 🤖 **Low-level agent parameters (`num_agents, time_steps, max_speed, view_range`)**

These values configure the simulated FlyAgents that operate under the Planner during training.

Example:
```python
config["environment"]["agent"]["planner_agent"]["num_agents"] = 4
config["environment"]["agent"]["planner_agent"]["max_speed"] = 10.0
```

#### 📦 **`default_model_folder` — The FlyAgent model used by the Planner**
```python
config["environment"]["agent"]["planner_agent"]["default_model_folder"] = "models/SimpleFlyAgent/"
config["environment"]["agent"]["planner_agent"]["default_model_name"] = "best_reward"
```

This is an important Planner setting. It specifies which FlyAgent model is used as the Planner’s low-level controller during training and evaluation.

- During **training**, you can choose a *simple* FlyAgent (e.g., no collision avoidance, small network), which makes training faster.
- During **evaluation**, you may switch this to a more advanced FlyAgent (e.g., with collision avoidance enabled, different action structure).

This makes the Planner flexible, it learns the strategy while delegating movement details to whichever FlyAgent model you provide.

### 🛠 Modifying the Configuration from Within the Notebook

The `config` object is just a nested Python dictionary.  
You can change values directly in the notebook before writing out the execution config.

Below are some common examples.

---

#### Change the config to your likings

```python
# Use NoGUI RL mode
config["settings"]["mode"] = 0  # 0 = GUI_RL # 2 = NoGUI_RL # 1/4 C++ Sim only

# Set the directory where models will be saved/loaded
config["paths"]["model_directory"] = "models/NotebookDemoRun"

# (Optional) Set a specific model to load.
# Leave as "" to let ROSHAN choose automatically (e.g. latest or best).
config["paths"]["model_name"] = ""

# Train a model
config["settings"]["rl_mode"] = "train"
# OR: Evaluate an existing model
config["settings"]["rl_mode"] = "eval"

# Use the low-level FlyAgent controller
config["settings"]["hierarchy_type"] = "fly_agent"
# OR: Use the high-level PlannerAgent
config["settings"]["hierarchy_type"] = "planner_agent"
```

In [2]:
# Use NoGUI RL mode
config["settings"]["mode"] = 2  # 2 = NoGUI_RL # 0 = GUI_RL

# Train the FlyAgent
config["settings"]["rl_mode"] = "train" # 'train' or 'eval'
config["settings"]["hierarchy_type"] = "fly_agent"

# Set model output directory
config["paths"]["model_directory"] = "models/NotebookDemoRun"
config["paths"]["model_name"] = ""

### ▶️ Executing ROSHAN with the Modified Configuration

Once your changes to the `config` dictionary are complete, the notebook writes the updated configuration to a separate file. This ensures that both Python and the underlying C++ simulator use the exact same parameters.

In [ ]:
notebook_exec_config_path = os.path.join(root_path, "notebook_config_exec.yaml")
with open(notebook_exec_config_path, 'w') as f:
    yaml.dump(config, f)
    
!python src/pysim/main.py "{notebook_exec_config_path}"

Using config file: /home/nex/Dokumente/Code/ROSHAN/notebook_config_exec.yaml
Mode: NoGUI_RL
Loading Firemodel without GUI
Using seed -537406765
Groundstation Start Percentage: 1
Imported all Config Parameters from/home/nex/Dokumente/Code/ROSHAN/used_config.yaml
Created ReinforcementLearning Handler
Running in NoGUI_RL mode. Agent always runs.
Created FireModel
Firemodel loaded

> openstreetmap@1.0.0 start
> nodemon server.js > server.log 2>&1 & echo $! > server.pid

Started OSM-Server with command status: 0
Loading map from: /home/nex/Dokumente/Code/ROSHAN/maps/35x36_Woods.tif
2025-12-02 15:12:06,131 - INFO - fly_agent - Logging File at /home/nex/Dokumente/Code/ROSHAN/models/NotebookDemoRun/logs/logging.log
2025-12-02 15:12:06,131 - INFO - fly_agent - Network sources loaded from /home/nex/Dokumente/Code/ROSHAN/src/pysim/networks/network_fly.py
2025-12-02 15:12:06,644 - INFO - fly_agent - Network sources archived at /home/nex/Dokumente/Code/ROSHAN/models/NotebookDemoRun/networks
2025-12